In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature
import copy
from constraint_relax import search_mfs
from item_extrator import data_path, dummy_data_path

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        if constraint != None:
            constraint_list.append(constraint)
    cl_set = set(constraint_list)
    problem.constraint_list = set(constraint_list)
    problem_list.append(problem)

In [4]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

sol_list = []
threshold_list = [0, 0.5, 1]
for problem in problem_list:
    prob_sol_list = []
    p1 = copy.deepcopy(problem) # original constraint
    prob_sol_list.append(p1.constraint_list)
    prob_sol_list.append(get_df_len(p1.solve()))

    for threshold in threshold_list:
        p = copy.deepcopy(problem)
        mfs = search_mfs(p.constraint_list, copy.copy(p.constraint_list), data_path)
        p.relax(mfs, threshold)
        prob_sol_list.append(p.constraint_list)
        prob_sol_list.append(get_df_len(p.solve()))
    sol_list.append(prob_sol_list)
result = pd.DataFrame(sol_list)

In [5]:
result

,0,1,2,3,4,5,6,7
0,"{storage: 512.0: 5: Nature.MORE, cpu: i7: 5: N...",0,"{os: 10: 5: Nature.EQUAL, storage: 512.0: 5: N...",1,"{storage: 512.0: 5: Nature.MORE, ram: 32: 3: N...",1,"{storage: 512.0: 5: Nature.MORE, ram: : 3: Nat...",19
1,"{cpu: pentium: 3: Nature.EQUAL, os: os: 3: Nat...",0,"{brand: microsoft: 4: Nature.EQUAL, ram: 4.0: ...",0,"{price: : 1: Nature.LESS, brand: microsoft: 4:...",0,"{os: : 3: Nature.EQUAL, brand: microsoft: 4: N...",1
2,"{storage: 1024.0: 3: Nature.MORE, ram: 2.0: 1:...",0,"{storage: 512: 3: Nature.MORE, ram: 2.0: 1: Na...",6,"{storage: 512: 3: Nature.MORE, brand: lenovo|m...",6,"{os: : 3: Nature.EQUAL, brand: : 3: Nature.EQU...",3
3,"{camera: True: 4: Nature.EQUAL, cpu: ryzen 3: ...",0,"{camera: True: 4: Nature.EQUAL, ram: 16.0: 5: ...",6,"{storage: 256.0: 5: Nature.MORE, cpu: : 2: Nat...",26,"{ram: 16.0: 5: Nature.MORE, storage: 256.0: 5:...",26
4,"{price: 4965.89: 2: Nature.LESS, brand: asus|h...",10,{brand: asus|huawei|microsoft|msi: 3: Nature.E...,11,{brand: asus|huawei|microsoft|msi: 3: Nature.E...,12,"{weight: 1.3: 2: Nature.LESS, camera: True: 3:...",12
...,...,...,...,...,...,...,...,...
132,"{os: 11: 3: Nature.EQUAL, storage: 256.0: 3: N...",5,"{storage: 256.0: 3: Nature.MORE, os: 11: 3: Na...",9,"{brand: : 1: Nature.EQUAL, ram: 4.0: 4: Nature...",9,"{storage: 256.0: 3: Nature.MORE, price: 2911.6...",9
133,"{storage: 256.0: 2: Nature.MORE, cpu: i7|i5: 2...",5,"{storage: 256.0: 2: Nature.MORE, price: 7341.5...",6,"{weight: 1.44: 4: Nature.LESS, cpu: i7|i5: 2: ...",5,"{os: : 2: Nature.EQUAL, weight: 1.44: 4: Natur...",25
134,"{ram: 8.0: 3: Nature.MORE, storage: 64.0: 5: N...",23,"{weight: 1.45: 5: Nature.LESS, brand: hp|huawe...",24,"{camera: True: 3: Nature.EQUAL, storage: 64.0:...",24,"{storage: 64.0: 5: Nature.MORE, brand: hp|huaw...",25
135,"{price: 7399.0: 2: Nature.LESS, brand: acer|le...",0,"{price: 7949.0: 2: Nature.LESS, brand: acer|le...",0,{brand: acer|lenovo|microsoft|dell: 1: Nature....,1,{brand: acer|lenovo|microsoft|dell: 1: Nature....,1
